Copyright **`(c)`** 2023 Thomas Baracco `<s308722@studenti.polito.it>`  
[`https://github.com/baraccothomas/computational-intelligence`](https://github.com/baraccothomas/computational-intelligence)  
Free for personal or classroom use.

In [101]:
import numpy as np

from random import random
from collections import namedtuple
from functools import reduce
from queue import PriorityQueue, SimpleQueue, LifoQueue


In [102]:
PROBLEM_SIZE = 5
NUM_SETS = 10
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)

State = namedtuple('State', ['taken', 'not_taken'])

In [103]:
def goal_check(state):
    return np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for i in range(PROBLEM_SIZE)]),
        )
    )


def distance(state):
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )

In [104]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

In [116]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((distance(state), state))

counter = 0
_, current_state = frontier.get()

while not goal_check(current_state):
    counter += 1
    for action in current_state.not_taken:
        new_state = State(
            current_state.taken ^ {action}, current_state.not_taken ^ {action}
        )
        frontier.put((distance(new_state), new_state))

    _, current_state = frontier.get()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

Solved in 3 steps (3 tiles)


In [119]:
current_state

State(taken={0, 2, 4}, not_taken={1, 3, 5, 6, 7, 8, 9})